In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalMaxPool2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [2]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [3]:
base_model.trainable = False

In [4]:
x = base_model.output
x = GlobalMaxPool2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

In [5]:
model = Model(inputs = base_model.input , outputs = predictions)

In [6]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
datagen = ImageDataGenerator(
    rescale=1. /255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

In [8]:
train_datagen = datagen.flow_from_directory(
    "Datasets/11 Flowers/train",
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 2602 images belonging to 5 classes.


In [9]:
validation_datagen = datagen.flow_from_directory(
    "Datasets/11 Flowers/test",
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 104 images belonging to 5 classes.


In [ ]:
model.fit(
    train_datagen,
    epochs=10,
    validation_data=validation_datagen
)

C:\Users\munje\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
73/82 ━━━━━━━━━━━━━━━━━━━━ 6s 743ms/step - accuracy: 0.5292 - loss: 2.0691

In [ ]:
test_loss ,  test_acc = model.evaluate(validation_datagen)

In [ ]:
x_val , y_val = next(validation_datagen)
predictions = model.predict(x_val)

In [ ]:
predicted_labels = np.argmax(predictions , axis=1)

In [ ]:
class_labels = list(train_datagen.class_indices.keys())

n = 10

plt.figure(figsize=(20,6))
for i in range(n):
    ax = plt.subplot(2, n , i+1)
    plt.imshow(x_val[i])

    actual_label = class_labels[np.argmax(y_val[i])]
    predicted_label = class_labels[predicted_labels[i]]

    plt.title(f"Actual : {actual_label} \n Predicted: {predicted_label}")
    plt.axis('off')
plt.show()